### Thanks to [valley](http://https://www.kaggle.com/valleyzw) for his wonderful [notebook](https://www.kaggle.com/code/valleyzw/ubiquant-lgbm-baseline?scriptVersionId=89116394). I used it as a baseline. In the final version (ver. 47) I didn't use any additional data and supplemental training. CV/LB of the final version is 0.147/0.1437

# Prepare data

In [ ]:
import os
import gc
import sys
import joblib
import random
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
from argparse import Namespace
from collections import defaultdict
from scipy.signal import find_peaks

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GroupKFold, train_test_split, KFold

import lightgbm as lgb
from lightgbm import LGBMRegressor

import matplotlib.style as style
style.use('fivethirtyeight') 

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Configuration

In [ ]:
args = Namespace(
    train=False,
    optimize=False,
    train_l2=False,
    inference=True,
    seed=21,
    folds=5,
    workers=4,
    min_time_id=None, 
    holdout=False,
    cv_method="single",
    num_bins=16,
    holdout_size=100,
    outlier_threshold=0.005,
    trading_days_per_year=250,   # chinese stock market trading days per year (roughly)
    add_investment_id_model=False,
    data_path=Path("../input/ubiquant-parquet"),
    just_eda=True,
)
seed_everything(args.seed)

assert args.cv_method in {"single", "kfold", "group", "stratified", "time", "group_time", "time_range"}, "unknown cv method"
assert args.data_path.exists(), "data_path not exists"

### Load the data

In [ ]:
%%time
train = pd.read_parquet(args.data_path.joinpath("train_low_mem.parquet"))
# assert train.isnull().any().sum() == 0, "null exists."
# assert train.row_id.str.extract(r"(?P<time_id>\d+)_(?P<investment_id>\d+)").astype(train.time_id.dtype).equals(train[["time_id", "investment_id"]]), "row_id!=time_id_investment_id"
# assert train.time_id.is_monotonic_increasing, "time_id not monotonic increasing"

# Features EDA + processing

### Possible outliers

In [ ]:
df = train[["investment_id", "target"]].groupby("investment_id").target.mean()
upper_bound, lower_bound = df.quantile([1-args.outlier_threshold, args.outlier_threshold])
display(upper_bound, lower_bound)
ax = df.plot(figsize=(16, 8))
ax.axhspan(lower_bound, upper_bound, fill=False, linestyle="--", color="k")
plt.show()

outlier_investments = df.loc[(df>upper_bound)|(df<lower_bound)|(df==0)].index
_=pd.pivot(
    train.loc[train.investment_id.isin(outlier_investments), ["investment_id", "time_id", "target"]],
    index='time_id', columns='investment_id', values='target'
).plot(figsize=(16,12), subplots=True, sharex=True)

### Drop indexes with feature outliers: [notebook](https://www.kaggle.com/junjitakeshima/ubiquant-simple-lgbm-removing-outliers-en-jp)

In [ ]:
outlier_list = []
outlier_col = []

for col in (f"f_{i}" for i in range(300)):
    _mean, _std = train[col].mean(), train[col].std()
    
    temp_df = train.loc[(train[col] > _mean + _std * 70) | (train[col] < _mean - _std * 70)]
    temp2_df = train.loc[(train[col] > _mean + _std * 35) | (train[col] < _mean - _std * 35)]
    if len(temp_df) >0 : 
        outliers = temp_df.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(temp_df))
    elif len(temp2_df)>0 and len(temp2_df) <6 :
        outliers = temp2_df.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(temp2_df))

outlier_list = list(set(outlier_list))
train.drop(train.index[outlier_list], inplace=True)
print(len(outlier_col), len(outlier_list), train.shape)

### Drop outliers from train

In [ ]:
if args.min_time_id is not None:
    train = train.query("time_id>=@args.min_time_id").reset_index(drop=True)
    gc.collect()
    
train=train.loc[~train.investment_id.isin(outlier_investments)].reset_index(drop=True)
train.shape

### Strange behaviour of some features according to time id

### f_1

In [ ]:
def get_unique_total_values_ratio(df, feature, plot=False, return_result=False):
    df_date_group = df.groupby('time_id').agg({feature: [lambda x: len(x.unique()), 'count']})
    df_date_group.columns.set_levels(['len_unique','count'], level=1,inplace=True)
    df_date_group['unique_count_ratio'] = df_date_group[feature]['len_unique']/df_date_group[feature]['count']
    df_date_group['num_or_cat'] = df_date_group['unique_count_ratio'].apply(lambda x: 1 if x > 0.5 else 0)
    if plot:
        fig = plt.figure(figsize=(14, 6))
        ax = fig.add_subplot(111)
        plt.plot(df_date_group.index, df_date_group['unique_count_ratio'] , label="Number of unique investments")
    if return_result:
        return df_date_group['num_or_cat']

get_unique_total_values_ratio(train, 'f_1', plot=True)

### f_7

In [ ]:
num_or_cat = get_unique_total_values_ratio(train, 'f_7', plot=True, return_result=True)

### Add time group feature

In [ ]:
train.loc[(train['time_id'] < 910), 'group'] = 0
train.loc[(train['time_id'] >= 910) & (train['time_id'] < 970), 'group'] = 1
train.loc[(train['time_id'] >= 970) & (train['time_id'] < 1030), 'group'] = 2
train.loc[(train['time_id'] >= 1030) & (train['time_id'] < 1090), 'group'] = 3
train.loc[(train['time_id'] >= 1090) & (train['time_id'] < 1150), 'group'] = 4
train.loc[(train['time_id'] >= 1150), 'group'] = 5
train['group'] = train['group'].astype(np.int16)

### Add feature that shows if some features are categorical or numerical for the current time_id

In [ ]:
# add numerical/categorical flag
train = train.merge(num_or_cat, how='left', on='time_id')

### Some features are not stationary. Let's make a rank transformation by time_id to make them stationary

In [ ]:
train['f_74']  = train[['time_id', 'f_74']].groupby('time_id').rank(pct=True)
train['f_142']  = train[['time_id', 'f_142']].groupby('time_id').rank(pct=True)
train['f_63']  = train[['time_id', 'f_63']].groupby('time_id').rank(pct=True)

### Make combinations for some features and drop not necessary features.

In [ ]:
num_features = [c for c in train.columns if c.startswith('f_')]
cat_features = ['num_or_cat'] #, 'sub_section', 'main_section']
features = num_features + cat_features + ['time_id'] #, 'year', 'quarter', 'market_value']

# make feature combinations
combination_features = ["f_231-f_250", "f_118-f_280", "f_155-f_297", "f_25-f_237", "f_179-f_265", "f_119-f_270", "f_71-f_197", "f_21-f_65"]
for f in combination_features:
    f1, f2 = f.split("-")
    train[f] = train[f1] + train[f2]
features += combination_features

# drop unnecessary features
to_drop = ["f_148", "f_72", "f_49", "f_205", "f_228", "f_97", "f_262", "f_258"]
features = list(sorted(set(features).difference(set(to_drop))))
train = train.drop(columns=["row_id"] + to_drop)

features_backup = features.copy()
gc.collect()

### Reduce memory usage.

In [ ]:
%%time

train = reduce_mem_usage(train)
train[["investment_id", "time_id"]] = train[["investment_id", "time_id"]].astype(np.uint16)
gc.collect()

### Standard scale numerical features

In [ ]:
%%time

sscaler = StandardScaler()
features_to_scale = [c for c in train.columns if c.startswith('f_')]
train[features_to_scale] = sscaler.fit_transform(train[features_to_scale])

# Train

### Set scoring and training functions

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# TODO: replace with feval_pearsonr
def feval_rmse(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'rmse', rmse(y_true, y_pred), False

# https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302480
def feval_pearsonr(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'pearsonr', pearsonr(y_true, y_pred)[0], True

# https://www.kaggle.com/gogo827jz/jane-street-supervised-autoencoder-mlp?scriptVersionId=73762661
# weighted average as per Donate et al.'s formula
# https://doi.org/10.1016/j.neucom.2012.02.053
# [0.0625, 0.0625, 0.125, 0.25, 0.5] for 5 fold
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w)

def run(info):    
    # hyperparams from: https://www.kaggle.com/valleyzw/ubiquant-lgbm-optimization
    params = {
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'learning_rate':0.05,
        "objective": "regression",
        "metric": "rmse",
        'boosting_type': "gbdt",
        'verbosity': 0,
        'n_jobs': -1, 
        'seed': 42,
#         'lambda_l1': 0.03627602394442367, 
#         'lambda_l2': 0.43523855951142926, 
#         'num_leaves': 114, 
#         'feature_fraction': 0.9505625064462319, 
#         'bagging_fraction': 0.9785558707339647, 
#         'bagging_freq': 7, 
#         'max_depth': -1, 
#         'max_bin': 501, 
#         'min_data_in_leaf': 374,
        'n_estimators': 1000, 
    }
    
    y = train['target']
    train['preds'] = -1000
    scores = defaultdict(list)
    features_importance= []
    
    def run_lgbm(trn_ind, val_ind):
        train_dataset = lgb.Dataset(train.loc[trn_ind, features], y.loc[trn_ind], categorical_feature=cat_features)
        valid_dataset = lgb.Dataset(train.loc[val_ind, features], y.loc[val_ind], categorical_feature=cat_features)
        model = lgb.train(
            params,
            train_set = train_dataset, 
            valid_sets = [train_dataset, valid_dataset], 
            callbacks=[lgb.log_evaluation(50), lgb.early_stopping(stopping_rounds=50)],
            feval = feval_pearsonr
        )
        joblib.dump(model, f'lgbm_seed_42_{info}.pkl')
        preds = model.predict(train.loc[val_ind, features])
        train.loc[val_ind, "preds"] = preds
        rmse_score = rmse(y.loc[val_ind], preds)
        pearsonr_score = pearsonr(y.loc[val_ind], preds)[0]
        features_importance = pd.DataFrame({'feature': features, 'importance': model.feature_importance()})
        del train_dataset, valid_dataset
        gc.collect()
        return rmse_score, pearsonr_score, features_importance
        
    time_thresh = round(train.time_id.max() * 0.9)
    trn_ind, val_ind = train.time_id < time_thresh, train.time_id >= time_thresh
    print(f"train length: {trn_ind.sum()}, valid length: {val_ind.sum()}")
    rmse_score, pearsonr_score, features_importance = run_lgbm(trn_ind, val_ind)
    
#     train.filter(regex=r"^(?!f_).*").to_csv(f"preds_{info}.csv", index=False)
    return features_importance, scores

### Train with deafault params

In [ ]:
%%time

if args.train:
    info = ""
    features_importance, scores = run(info=info)
    df = train[["target", "preds", "time_id"]].query("preds!=-1000")
    pearsonr_by_time_id = df.groupby("time_id").apply(lambda x: pearsonr(x.target, x.preds)[0]).mean()
    print(f"lgbm rmse: {rmse(df.target, df.preds):.4f}, pearsonr: {pearsonr(df.target, df.preds)[0]:.4f}, pearsonr by time_id: {pearsonr_by_time_id:.4f}")

#     features_importance.to_csv(f"features_importance_{info}.csv", index=False)

    plt.figure(figsize=(16, 10))
    plt.subplot(1,2,1)
    sns.barplot(x="importance", y="feature", data=features_importance.sort_values('importance', ascending=False).head(50))
    plt.title(f'Head LightGBM Features {info}')
    plt.subplot(1,2,2)
    sns.barplot(x="importance", y="feature", data=features_importance.sort_values('importance', ascending=False).tail(50))
    plt.title(f'Tail LightGBM Features {info}')
    plt.tight_layout()
    plt.show()
    del df

# del train
# gc.collect()

# Optimize LGBM with Optuna

In [ ]:
import optuna
import warnings
warnings.filterwarnings("ignore", module="lightgbm")
from sklearn.metrics import mean_squared_error

# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    # https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302480
    def feval_pearsonr(y_pred, lgb_train):
        y_true = lgb_train.get_label()
        return 'pearsonr', pearsonr(y_true, y_pred)[0], True
    
    time_thresh = round(train.time_id.max() * 0.9)
    trn_ind, val_ind = train.time_id < time_thresh, train.time_id >= time_thresh
    
    X_train, y_train = train.loc[trn_ind, features], train.loc[trn_ind, 'target']
    X_val, y_val = train.loc[val_ind, features], train.loc[val_ind, 'target']
    
    dtrain = lgb.Dataset(X_train, y_train, categorical_feature=cat_features)
    dvalid = lgb.Dataset(X_val, y_val, categorical_feature=cat_features)

    param = {
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'objective': 'regression',
        'metric': 'rmse',
        'seed': 42,
        'verbosity': 0,
        'boosting_type': 'gbdt', # other options: rf, dart, goss
        'force_col_wise': False, # Use only with CPU devices
       
        'subsample_for_bin': 300000, # Number of data that sampled to construct feature discrete bins; setting this 
                                     # to larger value will give better training result but may increase train time
        'n_estimators': trial.suggest_int('n_estimators', 500, 1200),      
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 1e-1),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256), # Max number of leaves in one tree
        'max_bin': trial.suggest_int('max_bin', 32, 255), # Max number of bins that feature values will be 
                                                           # bucketed in. small number of bins may reduce training 
                                                           # accuracy but may deal with overfitting
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0), # Randomly select a subset of features 
                                                                               # if feature_fraction < 1.0
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0), # Randomly select part of data without 
                                                                               # resampling if bagging_fraction < 1.0
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7), # Perform bagging at every k iteration
        'min_data_in_leaf': trial.suggest_int('min_child_samples', 5, 100), # Minimal number of data in one leaf
                                                                            # aliases: min_child_samples, 
        'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-4, 1e-1), # Stop trying to split 
                                                                                               # leave if sum of it's
                                                                                               # hessian less than k
        'cat_smooth': trial.suggest_float('cat_smooth', 10.0, 100.0), # this can reduce the effect of noises in 
                                                                      # categorical features, especially for 
                                                                      # categories with few data
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'pearsonr', valid_name='valid_1')
    gbm = lgb.train(
        param, 
        train_set = dtrain, 
        valid_sets = [dtrain, dvalid], 
        feval = feval_pearsonr,
        callbacks = [lgb.log_evaluation(100),
                     pruning_callback]
    )

    y_pred = gbm.predict(X_val)
    corr = pearsonr(y_val, y_pred)[0]
    return corr


if args.optimize is True:
    study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=100), direction="maximize"
    )
    study.optimize(objective, timeout=10*3600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


# Stacking best LGBM models

### Get best parameters

In [ ]:
if args.train_l2 is True:
    lgb_params = study.trials_dataframe()
    lgb_params.to_pickle('LGBM_Optuna_params.pkl')
    lgb_params = pd.read_pickle('LGBM_Optuna_params.pkl').sort_values('value', ascending=False).head(5)

    param_cols = [c for c in lgb_params.columns if c.startswith('params_')]
    lgb_params = lgb_params[param_cols]

    best_params = list()

    for idx, row in lgb_params.iterrows():
        row_dict = {k[7:]: v for k, v in row.items()}
        row_dict['device'] = 'gpu'
        row_dict['gpu_platform_id'] = 0
        row_dict['gpu_device_id'] = 0
        row_dict['verbosity'] = 0
        row_dict['max_bin'] = int(row_dict['max_bin'])
        row_dict['bagging_freq'] = int(row_dict['bagging_freq'])
        row_dict['min_child_samples'] = int(row_dict['min_child_samples'])
        row_dict['n_estimators'] = int(row_dict['n_estimators'])
        row_dict['num_leaves'] = int(row_dict['num_leaves'])
        best_params.append(row_dict)

### Create metafeatures for the train set

In [ ]:
if args.train_l2 is True:
    groups_level2 = [1, 2, 3, 4, 5]

    # That is how we get target for the 2nd level dataset
    y_train_level2 = train.loc[train.group.isin(groups_level2), 'target']

    # Create 1st level feature matrix
    X_train, y_train = train[features], train['target']
    
    # And here we create 2nd level feature matrix, init it with zeros first
    X_train_level2 = np.zeros([y_train_level2.shape[0], len(best_params)+1])
    X_train_level2[:, len(best_params)] = y_train_level2

    meta_index_begin = 0
    meta_index_end = 0

    # Now fill `X_train_level2` with metafeatures
    for current_group in tqdm(groups_level2):
        # split data
        train_index = X_train.loc[train.group <  current_group].index
        test_index  = X_train.loc[train.group == current_group].index

        X_train_l2 = X_train.loc[train_index, :]
        X_test_l2 =  X_train.loc[test_index, :]

        y_train_l2 = y_train[train_index]
        y_test_l2 =  y_train[test_index]

        meta_index_end += y_test_l2.shape[0]

        print(f"===================== time group: {current_group} =====================")
        
        # predict metafeatures for each of LGBM regressors
        for i, params in enumerate(tqdm(best_params)):
            print(f"===================== model: {i} =====================")
            train_dataset = lgb.Dataset(X_train_l2, y_train_l2, categorical_feature=cat_features)
            model = lgb.train(
                params,
                train_set = train_dataset, 
                valid_sets = [train_dataset], 
                callbacks = [lgb.log_evaluation(100)],
                feval = feval_pearsonr
            )
#             joblib.dump(model, f'lgbr_{i+1}.pkl')
            
            pred = model.predict(X_test_l2)
            X_train_level2[meta_index_begin:meta_index_end, i] = pred

            del train_dataset, model, pred
            gc.collect()

        meta_index_begin = meta_index_end

    X_train_level2 = pd.DataFrame(X_train_level2, columns=[f'lgbr_{i+1}' for i in range(len(best_params))]+['target'])
    X_train_level2.to_pickle('LGBM_X_train_level2.pkl')
    
    # train every LGB model on the full dataset and save it
    for i, params in enumerate(tqdm(best_params)):
        print(f"===================== model: {i} =====================")
        train_dataset = lgb.Dataset(X_train, y_train, categorical_feature=cat_features)
        model = lgb.train(
            params,
            train_set = train_dataset, 
            valid_sets = [train_dataset], 
            callbacks = [lgb.log_evaluation(100)],
            feval = feval_pearsonr
        )
        joblib.dump(model, f'lgbr_{i+1}.pkl')

    del X_train, y_train
    gc.collect()

# Fit LinearRegression for the second layer

In [ ]:
if args.train_l2 is True:
#     X_train_level2 = joblib.load('../input/ubiquant-lgbm-models/LGBM_X_train_level2.pkl')
    X_train_level2 = joblib.load('LGBM_X_train_level2.pkl')

    X = X_train_level2[[c for c in X_train_level2.columns if c != 'target']]
    y = X_train_level2['target']

    lr = LinearRegression()
    lr.fit(X, y)

    joblib.dump(lr, 'lr.pkl')

# Inference

In [ ]:
import ubiquant

time_id = 1220

if args.inference:
    env = ubiquant.make_env()  
    iter_test = env.iter_test()

    lr_model = joblib.load('../input/ubiquant-lgbm-models/lr.pkl')
    lgbr_models = list(map(joblib.load, sorted(Path("../input/ubiquant-lgbm-models").glob("lgbr_*.pkl"))))

    # https://www.kaggle.com/c/ubiquant-market-prediction/discussion/305353 
    # When making predictions check if the investment_id was in the train set
    for (test_df, sample_prediction_df) in iter_test:
        # get feature combinations
        for f in combination_features:
            f1, f2 = f.split("-")
            test_df[f] = test_df[f1] + test_df[f2]

        try:
            test_time_id = int(test_df['row_id'].values[0].split('_')[0]) # extract time_id from row_id
            test_df["time_id"] = test_time_id
        except:
            test_df["time_id"] = time_id
            test_time_id = 0
        # in case of error just increase time_id on 1
        if test_time_id:
            time_id = test_time_id + 1
        else:
            test_df["time_id"] = time_id
            time_id += 1
        
        # create num_or_cat feature for the test_df
        unqiues = len(test_df['f_7'].unique())
        if unqiues/len(test_df) > 0.5:
            test_df['num_or_cat'] = 0
        else:
            test_df['num_or_cat'] = 1
        
        # standard scale numeric features
        test_df[features_to_scale] = sscaler.transform(test_df[features_to_scale])
        
        # predict
        test_df["preds"] = lr_model.predict(np.stack([model.predict(test_df[features_backup]) for model in lgbr_models], axis=1))
        sample_prediction_df['target'] = test_df["preds"]
        env.predict(sample_prediction_df) 
        display(sample_prediction_df)

# Further ideas


- f_165, f_197, f_76, f_237, f_206 - correlate with target lag 1
- f_259, f_69, f_21, f_130, f_81   - correlate with target lag 2 (f_259 has the highes corr)
- f_149, f_233, f_110, f_42, f_105 - correlate with target lag 3


- try custom loss function (for DNN) (https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302181)

# What works for LGBM CV

- define if feature is numerical of categorical at this time period (num_or_cat)
- clipping of target (at 0.5% min/max)